In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.master('local').appName('initDataFrames').enableHiveSupport().getOrCreate()

In [3]:
spark

In [4]:
sc=spark.sparkContext

In [5]:
orders = spark.read.csv('data//retail_db//orders.csv')

### Creation of Dataframes in PySpark - Method 1

In [6]:
spark.createDataFrame([(1,)]).show()
#Creation of dataframes are enabled by list of tuples. Note that while mentioning tuples with one element then we need to specify comma at the end as above.

+---+
| _1|
+---+
|  1|
+---+



#### Using List

In [7]:
l = [('Alice', 1)]
spark.createDataFrame(l).show()

+-----+---+
|   _1| _2|
+-----+---+
|Alice|  1|
+-----+---+



In [8]:
spark.createDataFrame(l, ['name', 'age']).show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



#### Using Dictionary

In [9]:
d = [{'name': 'Alice', 'age': 1}]

In [10]:
spark.createDataFrame(d).show()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---+-----+
|age| name|
+---+-----+
|  1|Alice|
+---+-----+



#### Using rdd

In [11]:
l = [('Alice', 1)]
rdd = sc.parallelize(l)
spark.createDataFrame(rdd).show()

+-----+---+
|   _1| _2|
+-----+---+
|Alice|  1|
+-----+---+



In [12]:
spark.createDataFrame(rdd, ['name', 'age']).show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



In [13]:
spark.createDataFrame(rdd, "a: string, b: int").show()

+-----+---+
|    a|  b|
+-----+---+
|Alice|  1|
+-----+---+



#### Using Row

In [14]:
Person = Row('name', 'age')
person = rdd.map(lambda x: Person(*x))
spark.createDataFrame(person).show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



### Creation of Pyspark Dataframes - Method 2
#### By loading the file as RDD and explicitly manipulating the fields and specifying datatypes

In [15]:
ordersRDD = sc.textFile('data//retail_db//orders.csv')

In [16]:
orders = ordersRDD.map(lambda x:(Row(x.split(',')[0],x.split(',')[1],x.split(',')[2],x.split(',')[3]))).toDF(). \
        toDF('order_id','order_date','order_customer_id','order_status')
orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)



In [17]:
orders.show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|

### Creation of Pyspark Dataframes - Method 3
#### By loading the file as csv and mentioning the schema

In [18]:
ordersSchema = StructType([
StructField('order_id',IntegerType(),False),
StructField('order_date',TimestampType(),False),
StructField('order_customer_id',IntegerType(),False),
StructField('order_status',StringType(),False)
])
orders = spark.read.csv('data//retail_db//orders.csv',header=True,schema=ordersSchema)
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [19]:
orders.show()

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
|       6|2013-07-25 00:00:00|             7130|       COMPLETE|
|       7|2013-07-25 00:00:00|             4530|       COMPLETE|
|       8|2013-07-25 00:00:00|             2911|     PROCESSING|
|       9|2013-07-25 00:00:00|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|             1837|         CLOSED|
|      13|2013-07-25 00:0

#### By loading the file as csv and modify datatypes

In [20]:
orders = spark.read.csv('data//retail_db//orders.csv',header=True).toDF('order_id','order_date','order_customer_id','order_status')
orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)



In [21]:
orders.withColumn('order_id',orders.order_date.cast('int')).\
       withColumn('order_date',orders.order_date.cast('timestamp')).\
       withColumn('order_customer_id',orders.order_customer_id.cast('int')).\
       withColumn('order_status',orders.order_status.cast('string'))
orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)



In [22]:
orders.show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

#### By loading the file as csv and inferring schema automatically


In [23]:
orders = spark.read.csv('data//retail_db//orders.csv',header=True,inferSchema=True,sep=',')
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [24]:
orders.show()

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
|       6|2013-07-25 00:00:00|             7130|       COMPLETE|
|       7|2013-07-25 00:00:00|             4530|       COMPLETE|
|       8|2013-07-25 00:00:00|             2911|     PROCESSING|
|       9|2013-07-25 00:00:00|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|             1837|         CLOSED|
|      13|2013-07-25 00:0